# Scorer Evaluations - optional
You may want to run evaluations on PyRIT scorers to see how their scores align with human assessment. This notebook demonstrates how to do that using a dataset of sample assistant responses and associated human labels. There are two types of scorer evaluations: those for `objective` datasets and those for `harm` datasets. Scoring for whether a response has achieved an objective is based on a true/false basis while scoring for harmfulness of a response (given a certain harm category) is based on a float scale (0.0 to 1.0 where 0.0 is minimal severity and 1.0 is maximal), often on a Likert basis. Metrics produced for `harm` scoring evaluation include mean absolute error, one-sample t-test statistics, and Krippendorff's alpha (for interrater agreement) while metrics produced for `objective` scoring evaluation include accuracy, F1 score, precision, and recall. More detailed information on each of the metrics can be found in the `scorer_evaluator` module [here](../../../pyrit/score/scorer_evaluation/scorer_evaluator.py).

To actually run the evaluation in PyRIT, you need a `HumanLabeledDataset` object or a CSV file that includes columns of assistant responses, harm category or objectives (depending on the type of dataset you have), and human scores. You can then instantiate a PyRIT `Scorer` that you want to evaluate and pass it into a `HarmScorerEvaluator` or `ObjectiveScorerEvaluator` (which are subclasses of `ScorerEvaluator`). A `ScorerEvaluator` is responsible for running the end-to-end evaluation, and you can modify the number of model scoring trials as you see fit for your experiment. Note that for now, datasets with mixed harm categories cannot be used for evaluation, while datasets with mixed objectives can.

You can also retrieve metrics from a specific evaluation or from an evaluation that was previously saved using the `get_scorer_metrics` method and passing in the name of the dataset (`dataset_name`) that the evaluation was run on.

## Running Harm Scorer Evaluation
The following cell demonstrates how to run a custom harm scoring evaluation on a `Scorer` using a dataset of LLM assistant responses and human labels stored as a CSV file. In the following example, we use a small sample `hate_speech` dataset of assistant responses and human scores and run a harm evaluation using `SelfAskLikertScorer` with a Likert scale for hate speech. Sample datasets and saved metrics can be found in the `datasets/score/scorer_evals` directory.

In [ ]:
from dataclasses import asdict

from pyrit.common.path import SCORER_EVALS_HARM_PATH
from pyrit.prompt_target import OpenAIChatTarget
from pyrit.score import (
    LikertScalePaths,
    MetricsType,
    ScorerEvaluator,
    SelfAskLikertScorer,
)
from pyrit.setup import IN_MEMORY, initialize_pyrit

initialize_pyrit(memory_db_type=IN_MEMORY)

target = OpenAIChatTarget()
likert_scorer = SelfAskLikertScorer(chat_target=target, likert_scale_path=LikertScalePaths.HATE_SPEECH_SCALE.value)

# factory method that creates an HarmScorerEvaluator in this case since metrics_type is HARM.
evaluator = ScorerEvaluator.from_scorer(scorer=likert_scorer, metrics_type=MetricsType.HARM)
csv_path = f"{str(SCORER_EVALS_HARM_PATH)}/SAMPLE_hate_speech.csv"
# Uncomment the line below to use the full dataset of approx 200 entries
# csv_path = f"{str(SCORER_EVALS_HARM_PATH)}/hate_speech.csv"

# The dataset_name defaults to "SAMPLE_hate_speech" but can be manually set to any string. Each response is graded once by the scorer.
metrics = await evaluator.run_evaluation_from_csv_async( #type:ignore
    csv_path=csv_path, 
    assistant_response_col_name="assistant_response", 
    human_label_col_names=["human_score_1", "human_score_2", "human_score_3"], 
    objective_or_harm_col_name= "category", 
    num_scorer_trials=1,
)

# Metrics are saved to datasets/score/scorer_evals/harm/SAMPLE_hate_speech_SelfAskLikertScorer_metrics.json
# Results from the model scoring trials are saved to datasets/score/scorer_evals/harm/SAMPLE_hate_speech_SelfAskLikertScorer_scoring_results.csv
asdict(metrics)

{'mean_absolute_error': np.float64(0.15555555555555556),
 'mae_standard_error': np.float64(0.03226689113277704),
 't_statistic': np.float64(-0.6339667779924166),
 'p_value': np.float64(0.5363239606383386),
 'krippendorff_alpha_combined': np.float64(0.6613005332988628),
 'krippendorff_alpha_humans': np.float64(0.6594786524638434),
 'krippendorff_alpha_model': None}

## Retrieving Metrics
You can retrieve the metrics from the above evaluation by calling the `get_scorer_metrics` from the `ScorerEvaluator` class or directly from the `Scorer` class and passing in the `dataset_name` (which in this case is `SAMPLE_hate_speech`). This will throw an error if evaluation has not yet been run on that dataset. 

In [ ]:
# Either work for fetching the hate_speech metrics
evaluator.get_scorer_metrics(dataset_name = "SAMPLE_hate_speech")
likert_scorer.get_scorer_metrics(dataset_name = "SAMPLE_hate_speech", metrics_type=MetricsType.HARM)

# Retrieve metrics for the full hate_speech dataset that have already been computed and saved by the PyRIT team.
# full_metrics = likert_scorer.get_scorer_metrics(dataset_name="hate_speech")

HarmScorerMetrics(mean_absolute_error=0.16111111111111112, mae_standard_error=0.03188198402467754, t_statistic=-0.9509501669886251, p_value=0.3577591133678295, krippendorff_alpha_combined=0.6752950777253637, krippendorff_alpha_humans=0.6594786524638434, krippendorff_alpha_model=None)

## Running Objective Scorer Evaluation
The following cell demonstrates how to run a custom objective evaluation on a `Scorer` using a dataset of LLM assistant responses and human labels stored as a CSV file. This is much like the previous example except we use the `SelfAskRefusalScorer` that simply determines whether the model response was or was not a refusal.

In [ ]:
from pyrit.common.path import SCORER_EVALS_OBJECTIVE_PATH
from pyrit.score import SelfAskRefusalScorer

target = OpenAIChatTarget()
refusal_scorer = SelfAskRefusalScorer(chat_target=target)

# factory method that creates an ObjectiveScorerEvaluator in this case because the refusal scorer is a true/false scorer.
evaluator = ScorerEvaluator.from_scorer(scorer=refusal_scorer)
csv_path = f"{str(SCORER_EVALS_OBJECTIVE_PATH)}/SAMPLE_mixed_objective_refusal.csv"
# Uncomment the line below to use the full dataset of approx 200 entries
# csv_path = f"{str(SCORER_EVALS_OBJECTIVE_PATH)}/mixed_objective_refusal.csv"

# assistant_response_data_type_col_name is optional and can be used to specify the type of data for each response in the assistant response column.
metrics = await evaluator.run_evaluation_from_csv_async( #type:ignore
    csv_path=csv_path, 
    assistant_response_col_name="assistant_response", 
    human_label_col_names=["human_score"], 
    objective_or_harm_col_name= "objective",
    assistant_response_data_type_col_name="data_type", 
    num_scorer_trials=1,
)

refusal_scorer.get_scorer_metrics(dataset_name="SAMPLE_mixed_objective_refusal")

# Retrieve metrics for the full refusal scorer dataset that have already been computed and saved by the PyRIT team.
# full_metrics = likert_scorer.get_scorer_metrics(dataset_name="mixed_objective_refusal")

ObjectiveScorerMetrics(accuracy=1.0, accuracy_standard_error=0.0, f1_score=1.0, precision=1.0, recall=1.0)

In this case we use a `true_false_scorer` which determines whether the responses are harmful or not according to their harm category.

In [ ]:
from pathlib import Path

from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.common.path import SCORER_EVALS_OBJECTIVE_PATH
from pyrit.prompt_target import OpenAIChatTarget
from pyrit.score import (
    ObjectiveScorerEvaluator,
    ScorerEvaluator,
    SelfAskTrueFalseScorer,
)
from pyrit.score.true_false.self_ask_true_false_scorer import TRUE_FALSE_QUESTIONS_PATH

initialize_pyrit(memory_db_type=IN_MEMORY)

target = OpenAIChatTarget()

# Map harm categories to their respective dataset file paths and evaluation rubric file paths
# This allows for easy extension to new harm categories by or editing entries in this map.
# This also allows for easy editing of dataset file paths and evaluation rubric file paths without changing code logic.
harm_category_map = {
    "exploit": {
        "dataset_file_path": f"{str(SCORER_EVALS_OBJECTIVE_PATH)}/evaluation_datasets_10_21_2025/exploit_eval_dataset.csv",
        "dataset_name": "exploit_eval_dataset",
        "evaluation_rubric_file_path": str(Path(TRUE_FALSE_QUESTIONS_PATH, "exploits.yaml")),
    },
    "hate_speech": {
        "dataset_file_path": f"{str(SCORER_EVALS_OBJECTIVE_PATH)}/evaluation_datasets_10_21_2025/hate_speech_eval_dataset.csv",
        "dataset_name": "hate_speech_eval_dataset",
        "evaluation_rubric_file_path": str(Path(TRUE_FALSE_QUESTIONS_PATH, "hate_speech.yaml")),
    },
    "information_integrity": {
        "dataset_file_path": f"{str(SCORER_EVALS_OBJECTIVE_PATH)}/evaluation_datasets_10_21_2025/information_integrity_eval_dataset.csv",
        "dataset_name": "information_integrity_eval_dataset",
        "evaluation_rubric_file_path": str(Path(TRUE_FALSE_QUESTIONS_PATH, "information_integrity.yaml")),
    },
    "privacy": {
        "dataset_file_path": f"{str(SCORER_EVALS_OBJECTIVE_PATH)}/evaluation_datasets_10_21_2025/privacy_eval_dataset.csv",
        "dataset_name": "privacy_eval_dataset",
        "evaluation_rubric_file_path": str(Path(TRUE_FALSE_QUESTIONS_PATH, "privacy.yaml")),
    },
    "self-harm": {
        "dataset_file_path": f"{str(SCORER_EVALS_OBJECTIVE_PATH)}/evaluation_datasets_10_21_2025/self_harm_eval_dataset.csv",
        "dataset_name": "self_harm_eval_dataset",
        "evaluation_rubric_file_path": str(Path(TRUE_FALSE_QUESTIONS_PATH, "self-harm.yaml")),
    },
    "sexual_content": {
        "dataset_file_path": f"{str(SCORER_EVALS_OBJECTIVE_PATH)}/evaluation_datasets_10_21_2025/sexual_content_eval_dataset.csv",
        "dataset_name": "sexual_content_eval_dataset",
        "evaluation_rubric_file_path": str(Path(TRUE_FALSE_QUESTIONS_PATH, "sexual_content.yaml")),
    },
    "violence": {
        "dataset_file_path": f"{str(SCORER_EVALS_OBJECTIVE_PATH)}/evaluation_datasets_10_21_2025/violence_eval_dataset.csv",
        "dataset_name": "violence_eval_dataset",
        "evaluation_rubric_file_path": str(Path(TRUE_FALSE_QUESTIONS_PATH, "violence.yaml")),
    },
}

# set this list to the categories you want to evaluate
harm_categories_to_evaluate = ["information_integrity", "self-harm"]

for harm_category in harm_categories_to_evaluate:
    if harm_category not in harm_category_map:
        raise ValueError(
            f"Harm category '{harm_category}' not found in harm_category_map. Please add it to the map with the appropriate dataset and rubric file paths."
        )
    eval_rubric_path = harm_category_map[harm_category]["evaluation_rubric_file_path"]
    csv_path = str(Path(harm_category_map[harm_category]["dataset_file_path"]))
    dataset_name = harm_category_map[harm_category]["dataset_name"]

    true_false_scorer = SelfAskTrueFalseScorer(true_false_question_path=Path(eval_rubric_path), chat_target=target)

    evaluator: ObjectiveScorerEvaluator = ScorerEvaluator.from_scorer(scorer=true_false_scorer)  # type: ignore

    # assistant_response_data_type_col_name is optional and can be used to specify the type of data for each response in the assistant response column.
    metrics = await evaluator.run_evaluation_from_csv_async(  # type:ignore
        csv_path=csv_path,
        assistant_response_col_name="assistant_response",
        human_label_col_names=["human_score"],
        objective_or_harm_col_name="objective",
        assistant_response_data_type_col_name="data_type",
        num_scorer_trials=1,
    )

    print("Evaluation for harm category:", harm_category)
    # Use evaluator.get_scorer_metrics() instead of scorer.get_scorer_metrics()
    # The evaluator method accepts csv_path parameter
    print(evaluator.get_scorer_metrics(dataset_name=dataset_name, csv_path=csv_path))

Evaluation for harm category: information_integrity
ObjectiveScorerMetrics(accuracy=0.8888888888888888, accuracy_standard_error=0.060481228216868604, f1_score=0.896551724137931, precision=0.8125, recall=1.0)
Evaluation for harm category: self-harm
ObjectiveScorerMetrics(accuracy=0.5217391304347826, accuracy_standard_error=0.10415861863104658, f1_score=0.3529411764705882, precision=0.5, recall=0.2727272727272727)
